# Project1

The bank robber algorithm

--------------

You are a bank robber who's looking to rob as many banks in a day before you flee the country.

You got your hands on a list of banks in the area, with their location, the amount of money they have and the time they will take to rob. It looks like this:

```
id, x_coordinate, y_coordinate, money, time (hr)
0, 11.4, 3.3, 5000, 0.6
1, 6.9, 7.1, 15000, 0.3
2, 1.4, 13.2, 900, 1.1
```

This list of banks is in `bank_data.csv`

You have **24 hours** to make as much money as possible then escape.

# Rules:

- Your run can start anywhere on the map but it has to end at the **helicopter escape zone**: coordinates (0,0)
    - If you try to rob too many banks and can't get to the helicopter in 24 hours, you get caught and go to jail.

- You solution is a list or array of integers (eg. `[580, 433, 24, 998]`) where the numbers are the IDs of each banks. The ID of each bank is their index (their row index).

- You travel between banks at 30 km/h. You have to travel from one bank to the next!
    - Remember the formula to calculate the distance between two points.
    - The coordinates are in kilometers.
        - So (1, 1) and (1, 2) are one kilometer apart. 
        - This would take 1 / 30 hour = 2 minutes to travel

- Your solution should be an approximative/heuristic algorithm
    - This problem is NP-Hard, you won't find a fast enough algorithm that has the perfect solution
    - It doesn't need to find the best solution
    - Find the best solution you can!

- Your solution has to run on a common laptop in under 3 minutes for the 10,000 cities dataset
    - You can use everything you want to achieve this:
        - Use numpy, pandas, functions, algorithms
        - You can use parallelism
        - Use optimied libraries (pandas, numba, scipy, etc.)
    - Test your code on **small subsets** of the data so they run fast
        - Then scale your code up to bigger chunks of the data

- Your input is a pandas dataframe with the bank data. Your output is a list of bank IDs in order that you rob them:

**Ex:**

```
df = pd.read_csv('bank_data.csv')
robber_algorithm(df)

# Output is a list of bank IDs
[OUTPUT] --> [664, 2341, 26, 998, 9583, 24, 1, 444, 6783]
```

# Checking Your Solution:

You can use the `check_solution` function from `check_solution.py` to test if your solution is valid and verify the score.

# Hints:

- Most of the design paradigms we saw in class will work for this:
    - Divide-and-conquer
    - Brute Force
    - Greedy Algorithm
    - Dynamic Programming
    - Backtracking
    - Breadth-first & Depth-first search
    - Some we haven't seen:
        - Branch & Bound
        - Prune & Search
 
# Start with something that's easier (brute-force or greedy algorithm) and then work towards a better design once it works.
 
 - Because there are too many banks at each step, you will need to select only some candidates to explore
 
 - If you find yourself doing many **Nearest neighbors** type queries, consider using a [KD-Tree](https://en.wikipedia.org/wiki/K-d_tree) or a Ball Tree to speed it up.
     - There are good implementations of KD-Trees and nearest neighbours in scipy, sklearn and [this library](https://github.com/lmcinnes/pynndescent)

- You can work your algorithm backwards (starting at the end and backing up to the starting point) or forwards (finding a starting point and looping until there is no time left). They will lead to different designs however


In [1]:
import pandas as pd
import math
import numpy as np

In [2]:
df = pd.read_csv('bank_data.csv')

In [3]:
# determine highest value banks
# money/total_time(time (hr) + time_to_H)

df['time_to_H'] = (np.hypot(df['x_coordinate'],df['y_coordinate']))/30
df['rate1'] = df['money'] / df['time (hr)']
df['rate2'] = df['money'] / (df['time (hr)']+df['time_to_H'])
ndf1 = df.sort_values(by=['rate1'],ascending=False)
ndf2 = df.sort_values(by=['rate2'],ascending=False)
ndf3 = df.sort_values(by=['time_to_H'],ascending=True)

In [4]:
ndf1

,id,x_coordinate,y_coordinate,money,time (hr),time_to_H,rate1,rate2
3613,3613,-1.950527,-1.495858,54400,0.000186,0.081936,2.929603e+08,6.624322e+05
9546,9546,-1.810721,-0.795421,75100,0.000788,0.065924,9.530556e+07,1.125730e+06
3803,3803,3.867586,0.918332,18500,0.000358,0.132504,5.169381e+07,1.392425e+05
6528,6528,-0.282937,3.224334,33900,0.000769,0.107891,4.407762e+07,3.119826e+05
9583,9583,3.393835,-2.790830,24000,0.001131,0.146465,2.121396e+07,1.626054e+05
...,...,...,...,...,...,...,...,...
1838,1838,-2.179862,-3.064824,100,1.486872,0.125366,6.725528e+01,6.202558e+01
8832,8832,1.934915,-4.278272,100,1.490015,0.156516,6.711343e+01,6.073376e+01
4764,4764,0.565196,1.437413,100,1.491462,0.051485,6.704829e+01,6.481104e+01
2123,2123,-3.105083,-1.135305,100,1.494467,0.110204,6.691347e+01,6.231805e+01


In [5]:
ndf2

,id,x_coordinate,y_coordinate,money,time (hr),time_to_H,rate1,rate2
5135,5135,-0.404042,-0.433493,73900,0.009476,0.019753,7.799032e+06,2.528345e+06
8562,8562,0.928673,-0.846349,79100,0.008579,0.041883,9.219985e+06,1.567521e+06
433,433,-1.585902,0.009662,85300,0.016018,0.052864,5.325390e+06,1.238350e+06
9546,9546,-1.810721,-0.795421,75100,0.000788,0.065924,9.530556e+07,1.125730e+06
6104,6104,-0.943787,1.194650,94100,0.047608,0.050749,1.976579e+06,9.567229e+05
...,...,...,...,...,...,...,...,...
6865,6865,-3.766534,-3.804124,100,1.484619,0.178444,6.735734e+01,6.012999e+01
3903,3903,-4.865099,-2.759893,100,1.484204,0.186447,6.737618e+01,5.985691e+01
9849,9849,-4.163061,3.997267,100,1.482965,0.192380,6.743245e+01,5.968916e+01
7687,7687,-4.167891,4.522109,100,1.476305,0.204995,6.773667e+01,5.947777e+01


In [6]:
ndf3

,id,x_coordinate,y_coordinate,money,time (hr),time_to_H,rate1,rate2
4765,4765,0.038429,0.039438,12900,1.241626,0.001835,10389.598126,10374.261878
3936,3936,-0.000257,-0.068866,1100,0.474144,0.002296,2319.971153,2308.793263
8789,8789,0.084917,0.003973,93200,0.464666,0.002834,200574.197761,199358.455866
1212,1212,0.012560,0.094192,40100,0.290822,0.003168,137885.026828,136399.413639
5435,5435,-0.073901,-0.090091,36000,0.919935,0.003884,39133.207960,38968.676300
...,...,...,...,...,...,...,...,...
8110,8110,4.905826,-4.958823,10700,1.437018,0.232515,7445.975586,6408.976993
7831,7831,-4.946723,4.934850,500,1.381088,0.232911,362.033446,309.789546
1988,1988,4.978652,-4.904275,1600,1.263261,0.232949,1266.563466,1069.368445
1042,1042,-4.981968,4.917551,300,1.400481,0.233339,214.212061,183.618733


In [7]:
# initialise start position (first bank)
# loop the rows in the file
# while looping, compare the position and get distance between i, i-1

# start = first bank
# from first to next bank (i+1), time (to rob) + time (commute) = total time, if total time is < 24 go to next bank (add money as you move through to total amount collected)
# and keep track of distance from 0,0, check if remaining time is greater than time required to return to 0,0 from currrent position

In [8]:
# rate based on money/time_to_rob

rate1_arr=ndf1.to_numpy()

time_remaining = 23.9
total_time = 0
t = 0
loot = 0
bank_list = []
travel_tonext = 0

for i in range(len(rate1_arr)):
    if time_remaining >= 0 and rate1_arr[i][5] <= time_remaining:
        t = rate1_arr[i][4]
        travel_tonext = (np.hypot(((rate1_arr[i+1][1])-(rate1_arr[i][1])),((rate1_arr[i+1][2])-(rate1_arr[i][2]))))/30
        time_remaining -= t+travel_tonext
        total_time += t+travel_tonext
        loot += rate1_arr[i][3]
        bank_list.append(int(rate1_arr[i][0]))

print(time_remaining)
print(total_time)
print(loot)
print(bank_list)
print(len(bank_list))

-0.023891674498843024
23.92389167449885
6402400.0
[3613, 9546, 3803, 6528, 9583, 5933, 9195, 4757, 8550, 4499, 9241, 7343, 3798, 7544, 4762, 8436, 8562, 6097, 487, 2928, 5135, 865, 3914, 4987, 3466, 7560, 8579, 8690, 8469, 433, 524, 6468, 1914, 1757, 8287, 7258, 4725, 8206, 4605, 7064, 3926, 9401, 9736, 2331, 9653, 7074, 2346, 8966, 4696, 8022, 4789, 6740, 9881, 70, 2656, 951, 209, 58, 9228, 3193, 8525, 5610, 2037, 4287, 6022, 2827, 8849, 9378, 6104, 2741, 7701, 8355, 5563, 3005, 7665, 6987, 5719, 2729, 1997, 8286, 4906, 613, 790, 7087, 5126, 5562, 3516, 2243, 6759, 3026, 1372, 2442, 1733, 8375, 7764, 7649, 7689, 9880, 3297, 4794, 3089, 7583, 5725, 517, 4345, 2521, 2190, 5155, 9049, 444, 6712, 8908, 8703, 7877, 7595, 8231, 9275, 4465, 5295, 5296, 299, 8846, 6254, 507, 6535, 6216, 279, 6375]
128


In [9]:
# rate based on money/(time_to_rob + time_to_H)

rate2_arr=ndf2.to_numpy()

time_remaining = 23.8
total_time = 0
t = 0
loot = 0
bank_list = []
travel_tonext = 0

for i in range(len(rate2_arr)):
    if time_remaining >= 0 and rate2_arr[i][5] <= time_remaining:
        t = rate2_arr[i][4]
        travel_tonext = (np.hypot(((rate2_arr[i+1][1])-(rate2_arr[i][1])),((rate2_arr[i+1][2])-(rate2_arr[i][2]))))/30
        time_remaining -= t+travel_tonext
        total_time += t+travel_tonext
        loot += rate2_arr[i][3]
        bank_list.append(int(rate2_arr[i][0]))

print(time_remaining)
print(total_time)
print(loot)
print(bank_list)
print(len(bank_list))

-0.02147581976384691
23.821475819763844
8241300.0
[5135, 8562, 433, 9546, 6104, 9653, 5933, 1455, 1372, 5719, 557, 7544, 6254, 951, 6097, 3613, 5356, 8525, 8287, 613, 3914, 7560, 7064, 5610, 4789, 4287, 487, 5971, 7595, 3516, 8375, 3605, 4696, 3193, 1447, 7494, 7531, 8125, 2, 9241, 9881, 4610, 8436, 8469, 781, 1966, 7087, 7265, 3026, 4906, 9378, 6317, 7583, 3340, 3971, 839, 6535, 2639, 58, 5155, 3466, 4499, 6478, 9779, 1757, 7689, 5296, 5622, 6759, 7074, 5562, 3005, 8690, 1844, 6375, 865, 6216, 4422, 7772, 517, 670, 2346, 4234, 3297, 7665, 1684, 6740, 5563, 2458, 1976, 3798, 6281, 2331, 9049, 359, 8355, 2190, 6468, 9804, 2243, 9880, 7907, 5381, 4492, 488, 2028, 2226, 8477, 4807, 1860, 9736, 7801, 4762, 4627, 8503, 6528]
116


In [10]:
# rate based on money/(time_to_rob + time_to_H)
# sorted for shortest time_to_H

rate3_arr=ndf3.to_numpy()

time_remaining = 23.8
total_time = 0
t = 0
loot = 0
bank_list = []
travel_tonext = 0

for i in range(len(rate3_arr)):
    if time_remaining >= 0 and rate3_arr[i][5] <= time_remaining:
        t = rate3_arr[i][4]
        travel_tonext = (np.hypot(((rate3_arr[i+1][1])-(rate3_arr[i][1])),((rate3_arr[i+1][2])-(rate3_arr[i][2]))))/30
        time_remaining -= t+travel_tonext
        total_time += t+travel_tonext
        loot += rate3_arr[i][3]
        bank_list.append(int(rate2_arr[i][0]))

print(time_remaining)
print(total_time)
print(loot)
print(bank_list)
print(len(bank_list))

-0.029257796157200477
23.829257796157204
502900.0
[5135, 8562, 433, 9546, 6104, 9653, 5933, 1455, 1372, 5719, 557, 7544, 6254, 951, 6097, 3613, 5356, 8525, 8287, 613, 3914, 7560, 7064, 5610, 4789, 4287, 487, 5971, 7595, 3516]
30
